In [3]:
#23/8/24 Creates a K fold KNN model for each file in a folder provided they are chemopy

import os
import pandas as pd
import numpy as np
import math
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Chemopy/datasets/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
resultslist = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 1
        maxval = 0
        for mtry in range(1, 4):
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = mtry,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)


            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = mtry
        #store best model for the given fold and plot the metric vs mcc value
        rf = SVM = svm.SVC(C=1, kernel='poly', gamma='scale',degree = bestmetrics,
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)

        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    resultslist.append(results[12])

For dataset NR-ER_Chemopy.csv
for fold 1 test set mcc of 0 valid set mcc of 0
for fold 2 test set mcc of 0 valid set mcc of 0
for fold 3 test set mcc of 0 valid set mcc of 0
for fold 4 test set mcc of 0.1588352020667356 valid set mcc of 0.15932444118273442
for fold 5 test set mcc of 0 valid set mcc of 0

validation metrics of:
positives in data 0
negatives in data 0
net accuracy = 0
mcc = 0
For dataset Developmental_Toxicity_Chemopy.csv
for fold 1 test set mcc of 0 valid set mcc of 0
for fold 2 test set mcc of 0 valid set mcc of 0
for fold 3 test set mcc of 0 valid set mcc of 0
for fold 4 test set mcc of 0 valid set mcc of 0
for fold 5 test set mcc of 0 valid set mcc of 0

validation metrics of:
positives in data 8
negatives in data 6
net accuracy = 0.5714285714285714
mcc = 0.05892556509887897
For dataset NR-aromatase_Chemopy.csv
for fold 1 test set mcc of 0.225 valid set mcc of 0.5509189647176967
for fold 2 test set mcc of 0.38012335774385503 valid set mcc of 0.5321940341812832
for fo

for fold 3 test set mcc of 0.39721346067661756 valid set mcc of 0.4065892220164691
for fold 4 test set mcc of 0.3777193916268797 valid set mcc of 0.3764842843465182
for fold 5 test set mcc of 0.36903498632951254 valid set mcc of 0.3146302577969878

validation metrics of:
positives in data 840
negatives in data 657
net accuracy = 0.6806947227788911
mcc = 0.3429471661974262
For dataset Hepatotoxicity_Chemopy.csv
for fold 1 test set mcc of 0.25324868678969253 valid set mcc of 0.3154819973181133
for fold 2 test set mcc of 0.2869180996538079 valid set mcc of 0.23222224417139567
for fold 3 test set mcc of 0.4067569831052704 valid set mcc of 0.24222622168630947
for fold 4 test set mcc of 0.33518510525424844 valid set mcc of 0.2574972667407181
for fold 5 test set mcc of 0.3018442564491325 valid set mcc of 0.34342860127135305

validation metrics of:
positives in data 293
negatives in data 284
net accuracy = 0.6377816291161178
mcc = 0.3113962919529771
For dataset NR-AR_Chemopy.csv
for fold 1 tes

In [7]:
namelist

['NR-ER_Chemopy.csv',
 'Developmental_Toxicity_Chemopy.csv',
 'NR-aromatase_Chemopy.csv',
 'SR-ATAD5_Chemopy.csv',
 'SR-ARE_Chemopy.csv',
 'SR-p53_Chemopy.csv',
 'Cardiotoxicity-1_Chemopy.csv',
 'SR-HSE_Chemopy.csv',
 'Respiratory_Toxicity_Chemopy.csv',
 'Carcinogenicity_Chemopy.csv',
 'Cardiotoxicity-30_Chemopy.csv',
 'Cardiotoxicity-10_Chemopy.csv',
 'Cardiotoxicity-5_Chemopy.csv',
 'NR-AR-LBD_Chemopy.csv',
 'Ames_Mutagenicity_Chemopy.csv',
 'Hepatotoxicity_Chemopy.csv',
 'NR-AR_Chemopy.csv',
 'Reproductive_Toxicity_Chemopy.csv',
 'NR-ER-LBD_Chemopy.csv',
 'NR-PPAR-gamma_Chemopy.csv',
 'NR-AhR_Chemopy.csv',
 'SR-MMP_Chemopy.csv']

In [6]:
len(resultslist)

22

In [8]:
temp = pd.DataFrame(resultslist, columns=['MCC'])
temp2 = pd.DataFrame(namelist, columns=['Dataset'])
resultframe = pd.concat([temp2, temp], axis=1)
resultframe.to_csv('/Users/james/Documents/Honours/Results/Figure_Development/Chemopy/resultsdata/chemopy_svm.csv')

In [1]:
#23/8/24 Creates a K fold KNN model for each file in a folder provided they are chemopy

import os
import pandas as pd
import numpy as np
import math
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sys
sys.path.insert(1, '/Users/james/Documents/Honours/Python')
from Function_Repo import metriccalc 
from Function_Repo import setbalance
import math

seed = 81
#Data import
datasets = []
index = []
directory = '/Users/james/Documents/Honours/Data/Chemopy/datasets/'
namelist = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if 'csv' in filename:
        pathname = directory + file
        df = pd.read_csv(pathname)
        df = setbalance(df)
        datasets.append(df)
        namelist.append(filename)

sets = 0
resultslist = []
for df in datasets:
    df = df.dropna()
    dfarray = df.to_numpy()
    
    dfarray, tempset = train_test_split(dfarray, test_size=0.2, 
                                        random_state=seed, stratify = df['Toxicity_Value'])
    
    size = len(dfarray) / 5
    splitsize = math.ceil(size)
    empty = [[] for _ in range(5)]
    splits = []
    for list in empty:
        splits.append(list)
        
    loops = 0
    currsplit = 0
    #Results in a list of 5 lists that each contain 1/5 of the targetdata
    for row in dfarray:
        splits[currsplit].append(row)
        if loops == splitsize:
            loops = 0
            currsplit = currsplit + 1
        loops = loops + 1

    folds = []
    for split in splits:
        temp = pd.DataFrame(split)
        folds.append(temp)
    
    model_list = []
    validlist = []
    totalmetrics = []
    testdat = []
    #For loop that uses each fold once for valid/testing and the rest for training
    #Each 'split' in the range corresponds to the set used for test/validation with the other 4 for training
    iteration = 1
    print('========================================')
    print('For dataset', namelist[sets])
    for split in range(0,5):
        #Set creation
        trainlist = [df for i, df in enumerate(folds) if i != split]
        trainset = pd.concat(trainlist, axis=0)
        testset, validset = train_test_split(folds[split], test_size=0.5, random_state=seed)

        ytrain = trainset[0].values
        xtrain = trainset.iloc[:, 1:]

        ytest = testset[0].values
        xtest = testset.iloc[:, 1:]

        yvalid = validset[0].values
        xvalid = validset.iloc[:, 1:]

        #lists of metric values
        mcclist = []
        bestmetrics = 'linear'
        maxval = 0
        mtry = ['linear', 'rbf']
        for item in mtry:
            #using mtry as the adjusted hyperparameter creates a series of random forests
            rf = SVM = svm.SVC(C=1, kernel=item, gamma='scale',
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)


            model= rf.fit(xtrain, ytrain)
            preds = model.predict(xtest)
            #calculate metric (mcc)
            mcc = metriccalc(preds, ytest)[12]
            mcclist.append(mcc)
            if mcc > maxval:
                maxval = mcc
                bestmetrics = item
        #store best model for the given fold and plot the metric vs mcc value
        rf = SVM = svm.SVC(C=1, kernel=bestmetrics, gamma='scale',
              coef0=0.4, shrinking=True, probability=False, 
              tol=0.001, cache_size=200, class_weight=None, 
              verbose=False, max_iter=-1, decision_function_shape='ovr', 
              break_ties=False, random_state=seed)

        model_list.append(rf)
        testdat.append(maxval)
        totalmetrics.append(mcclist)

        #check models onto validation set, printing various metrics
        model= rf.fit(xtrain, ytrain)
        preds = model.predict(xvalid)
        results = metriccalc(preds, yvalid)
        validlist.append(results)
        print('for fold', split + 1, 'test set mcc of', maxval, 'valid set mcc of', results[12])
        
        iteration = iteration + 1
        
        #get x and y values for the final validation set

    toxvals = []
    fingerprints = []
    for row in tempset:
        toxvals.append(row[0])
        fingerprints.append(row[1:])
        
    #calculates consensus of models on each fingerprint in the final validation set
    consensuslist = []
    predictions = []
    for fp in fingerprints:
        consensus = -1
        fp = fp.reshape(1, -1)
        #appends each model's prediction to a list
        predictions = []
        predlist = []
        for model in model_list:
            preds = model.predict(fp)
            predictions.append(preds)
            predlist.append(preds)
        predictions.append(predlist)

        #finds number of 0s in the prediction list
        zercount = 0
        for num in predictions:
            if num == 0:
                zercount = zercount + 1
        #as there are 5 models, if there are less than 3 0s predicted, the consensus is 1
        if zercount < 3 :
            consensus = 1
        else:
            consensus = 0
        consensuslist.append(consensus)
    #calculate and print metrics
    results = metriccalc(consensuslist, toxvals)
    print('\nvalidation metrics of:')
    print('positives in data', results[0])
    print('negatives in data', results[1])
    print('net accuracy =', results[6])
    print('mcc =',results[12])
    sets += 1
    resultslist.append(results[12])

For dataset NR-ER_Chemopy.csv
for fold 1 test set mcc of 0.5052772741042054 valid set mcc of 0.4602087751819045
for fold 2 test set mcc of 0.2792112497575975 valid set mcc of 0.29155014679337427
for fold 3 test set mcc of 0.4016532484263004 valid set mcc of 0.43792288332178286
for fold 4 test set mcc of 0.4154119855855661 valid set mcc of 0.22687304640656067
for fold 5 test set mcc of 0.266940563676722 valid set mcc of 0.3476569836199849

validation metrics of:
positives in data 154
negatives in data 209
net accuracy = 0.6776859504132231
mcc = 0.3286618503223067
For dataset Developmental_Toxicity_Chemopy.csv
for fold 1 test set mcc of 0 valid set mcc of 0
for fold 2 test set mcc of 0 valid set mcc of 0
for fold 3 test set mcc of 0 valid set mcc of 0
for fold 4 test set mcc of 0 valid set mcc of 0
for fold 5 test set mcc of 0 valid set mcc of 0

validation metrics of:
positives in data 8
negatives in data 6
net accuracy = 0.6428571428571429
mcc = 0.251259453814803
For dataset NR-aromata

for fold 5 test set mcc of 0.5530100413375021 valid set mcc of 0.4625

validation metrics of:
positives in data 45
negatives in data 49
net accuracy = 0.7553191489361702
mcc = 0.5316947973422216
For dataset Ames_Mutagenicity_Chemopy.csv
for fold 1 test set mcc of 0.4933777690178032 valid set mcc of 0.5964573894774567
for fold 2 test set mcc of 0.5043823942031785 valid set mcc of 0.5916760104029402
for fold 3 test set mcc of 0.5040039232019919 valid set mcc of 0.540390884921481
for fold 4 test set mcc of 0.5179183473206044 valid set mcc of 0.5249855932284696
for fold 5 test set mcc of 0.556299838457902 valid set mcc of 0.48143046650254334

validation metrics of:
positives in data 840
negatives in data 657
net accuracy = 0.7862391449565799
mcc = 0.5635108167627568
For dataset Hepatotoxicity_Chemopy.csv
for fold 1 test set mcc of 0.3128665543898515 valid set mcc of 0.3499832588397372
for fold 2 test set mcc of 0.3053124886337205 valid set mcc of 0.39403253727940196
for fold 3 test set mcc